In [ ]:
format ELF64 executable

segment executable readable
entry _start

_start:
    ; Получаем аргументы командной строки
    pop rcx         ; argc
    cmp rcx, 2
    jl exit         ; если нет аргументов
    
    pop rsi         ; argv[0] - имя программы
    pop rsi         ; argv[1] - наш символ
    
    ; Проверяем, что аргумент не пустой
    cmp byte [rsi], 0
    je exit
    
    ; Сохраняем символ
    mov al, [rsi]
    mov [char_buffer], al
    
    ; Выводим "символ "
    mov rax, 1
    mov rdi, 1
    mov rsi, symbol_msg
    mov rdx, symbol_len
    syscall
    
    ; Выводим сам символ
    mov rax, 1
    mov rdi, 1
    mov rsi, char_buffer
    mov rdx, 1
    syscall
    
    ; Выводим ", ASCII "
    mov rax, 1
    mov rdi, 1
    mov rsi, ascii_msg
    mov rdx, ascii_len
    syscall
    
    ; Преобразуем ASCII-код в строку
    movzx rax, byte [char_buffer]
    mov rdi, num_buffer
    call int_to_string
    
    ; Находим длину строки с числом
    mov rsi, num_buffer
    mov rdx, 0
.find_length:
    cmp byte [rsi + rdx], 0
    je .found
    inc rdx
    jmp .find_length
.found:
    
    ; Выводим ASCII-код
    mov rax, 1
    mov rdi, 1
    mov rsi, num_buffer
    syscall
    
    ; Новая строка
    mov rax, 1
    mov rdi, 1
    mov rsi, newline
    mov rdx, 1
    syscall
    
    jmp exit

; Функция преобразования числа в строку
int_to_string:
    push rbx
    mov rcx, 10
    mov rbx, 15
    mov byte [rdi + 16], 0
    
.convert:
    xor rdx, rdx
    div rcx
    add dl, '0'
    mov [rdi + rbx], dl
    dec rbx
    test rax, rax
    jnz .convert
    
.fill:
    cmp rbx, 0
    jl .done
    mov byte [rdi + rbx], ' '
    dec rbx
    jmp .fill
    
.done:
    pop rbx
    ret

exit:
    mov rax, 60     ; sys_exit
    xor rdi, rdi
    syscall

segment readable writeable
    symbol_msg db 'символ '
    symbol_len = $ - symbol_msg
    ascii_msg db ', ASCII '
    ascii_len = $ - ascii_msg
    newline db 10
    
    char_buffer rb 1
    num_buffer rb 16

@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./1 A
символ A, ASCII 65
@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./1 x
символ x, ASCII 120
@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./1 " "
символ  , ASCII 32

In [ ]:
format ELF64 executable

entry _start

segment readable executable

_start:
    ; Получаем аргументы командной строки
    pop rcx             ; argc
    cmp rcx, 4          ; проверяем, что есть 3 аргумента + имя программы
    jl exit             ; если аргументов меньше - выходим
    
    pop rsi             ; пропускаем argv[0] (имя программы)
    
    ; Читаем первый аргумент (a)
    pop rsi
    call str_to_int
    mov [a], rax
    
    ; Читаем второй аргумент (b)  
    pop rsi
    call str_to_int
    mov [b], rax
    
    ; Читаем третий аргумент (c)
    pop rsi
    call str_to_int
    mov [c], rax
    
    ; Вычисляем выражение: (((b*c) - b) + c) - a) - b
    mov rax, [b]
    imul qword [c]      ; rax = b * c
    sub rax, [b]        ; rax = (b*c) - b
    add rax, [c]        ; rax = ((b*c) - b) + c
    sub rax, [a]        ; rax = (((b*c) - b) + c) - a
    sub rax, [b]        ; rax = ((((b*c) - b) + c) - a) - b
    
    ; Выводим результат
    call print_int
    call new_line
    call exit

; Функция преобразования строки в число
str_to_int:
    push rbx
    push rcx
    push rdx
    xor rax, rax        ; обнуляем результат
    xor rbx, rbx        ; обнуляем для работы с байтами
    mov rcx, 10         ; основание системы счисления
    
.convert_loop:
    mov bl, [rsi]       ; берем текущий символ
    cmp bl, 0           ; конец строки?
    je .done
    cmp bl, '0'
    jl .done
    cmp bl, '9'
    jg .done
    
    sub bl, '0'         ; преобразуем символ в цифру
    imul rax, rcx       ; умножаем текущий результат на 10
    add rax, rbx        ; добавляем новую цифру
    
    inc rsi             ; переходим к следующему символу
    jmp .convert_loop
    
.done:
    pop rdx
    pop rcx
    pop rbx
    ret

; Функция вывода числа
print_int:
    push rbx
    push rcx
    push rdx
    
    mov rcx, 10
    xor rbx, rbx        ; счетчик цифр
    
    ; Проверяем отрицательное число
    test rax, rax
    jns .positive
    neg rax             ; делаем число положительным
    push rax
    mov al, '-'
    call print_symbl    ; выводим минус
    pop rax
    
.positive:
    ; Извлекаем цифры и сохраняем в стек
.iter1:
    xor rdx, rdx
    div rcx             ; rax / 10, остаток в rdx
    add dl, '0'         ; преобразуем цифру в символ
    push rdx
    inc rbx
    cmp rax, 0
    jne .iter1
    
    ; Выводим цифры из стека
.iter2:
    pop rax
    call print_symbl
    dec rbx
    jnz .iter2
    
    pop rdx
    pop rcx
    pop rbx
    ret

; Функция вывода одного символа
print_symbl:
    push rax
    push rdi
    push rsi
    push rdx
    
    ; Сохраняем символ в стеке
    push rax
    
    ; Системный вызов write
    mov rax, 1          ; sys_write
    mov rdi, 1          ; stdout
    mov rsi, rsp        ; указатель на символ в стеке
    mov rdx, 1          ; длина 1 байт
    syscall
    
    ; Восстанавливаем стек
    pop rax
    
    pop rdx
    pop rsi
    pop rdi
    pop rax
    ret

; Функция вывода новой строки
new_line:
    push rax
    push rdi
    push rsi
    push rdx
    
    mov rax, 1          ; sys_write
    mov rdi, 1          ; stdout
    mov rsi, newline
    mov rdx, 1          ; длина 1 байт
    syscall
    
    pop rdx
    pop rsi
    pop rdi
    pop rax
    ret

; Функция выхода
exit:
    mov rax, 60         ; sys_exit
    xor rdi, rdi        ; код возврата 0
    syscall

segment readable writeable
    newline db 10
    a dq 0
    b dq 0
    c dq 0

@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./2 2 3 4
8
@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./2 1 2 3
4
@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./2 5 5 5
10
@KoludarovaMaria ➜ /workspaces/System_programming (main) $ ./2 10 2 8
10

In [ ]:
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
    int a = atoi(argv[1]);
    int b = atoi(argv[2]);
    int c = atoi(argv[3]);
    int result = (((((b * c) - b) + c) - a) - b);
    printf(" %d\n", result);
    return 0;
}